In [1]:
from manim import *
from typing import Sequence

Manim Community v0.12.0

In [2]:
class Charge(VGroup):
    def __init__(self, magnitude=1, point=ORIGIN, add_glow=True, **kwargs):
        VGroup.__init__(self, **kwargs)
        self.magnitude = magnitude
        self.point = point
        self.radius = (abs(magnitude) * 0.4 if abs(magnitude) < 2 else 0.8) * 0.3

        if magnitude > 0:
            label = VGroup(
                Rectangle(width=0.32 * 1.1, height=0.006 * 1.1).set_z_index(1),
                Rectangle(width=0.006 * 1.1, height=0.32 * 1.1).set_z_index(1),
            )
            color = RED
            layer_colors = [RED_D, RED_A]
            layer_radius = 4
        else:
            label = Rectangle(width=0.27, height=0.003)
            color = BLUE
            layer_colors = ["#3399FF", "#66B2FF"]
            layer_radius = 2

        if add_glow:  # use many arcs to simulate glowing
            layer_num = 80
            color_list = color_gradient(layer_colors, layer_num)
            opacity_func = lambda t: 1500 * (1 - abs(t - 0.009) ** 0.0001)
            rate_func = lambda t: t ** 2

            for i in range(layer_num):
                self.add(
                    Arc(
                        radius=layer_radius * rate_func((0.5 + i) / layer_num),
                        angle=TAU,
                        color=color_list[i],
                        stroke_width=101
                        * (rate_func((i + 1) / layer_num) - rate_func(i / layer_num))
                        * layer_radius,
                        stroke_opacity=opacity_func(rate_func(i / layer_num)),
                    ).shift(point)
                )

        self.add(Dot(point=self.point, radius=self.radius, color=color))
        self.add(label.scale(self.radius / 0.3).shift(point))
        for mob in self:
            mob.set_z_index(1)


class ElectricField(ArrowVectorField):
    def __init__(self, *charges: Charge, **kwargs):
        self.charges = charges
        super().__init__(
            lambda p: self.field_func(p),
            **kwargs
        )

    def field_func(self, p):
        direction = np.zeros(3)
        pos = []
        for charge in self.charges:
            p0, mag = charge.get_center(), charge.magnitude
            pos.append(p0)
            x, y, z = p - p0
            dist = (x ** 2 + y ** 2) ** 1.5
            if any((p - p0) ** 2 > 0.05):
                direction += mag * np.array([x / dist, y / dist, 0])
        return direction

In [30]:
%%manim -qm -v WARNING ElectricFieldExampleScene

class ElectricFieldExampleScene(Scene):
    def construct(self):
        charge1 = Charge(2, LEFT)
        field = ElectricField(charge1)
        self.add(charge1)
        self.add(field)
        
        circle = Circle(radius=1, color=WHITE).shift(LEFT)
        self.play(FadeIn(circle))
        
        text1 = MathTex(r"\xrightarrow{x^6y^8}", font_size=72).shift(UR*2)
        text2 = Tex(r"$\xrightarrow{x^6y^8}$", font_size=72).next_to(text1, DOWN)
        self.add(VGroup(text1, text2))
        
        self.wait(1)
#         func = lambda p: field.field_func(p)
#         stream_lines = StreamLines(func, stroke_width=3, max_anchors_per_line=30)
#         self.add(stream_lines)
#         stream_lines.start_animation(warm_up=False, flow_speed=1.5)
#         self.wait(4)

In [3]:
%%manim -qh -v WARNING ElectricFieldExampleScene

class ElectricFieldExampleScene(Scene):
    def construct(self):
        charge1 = Charge(2, LEFT*2.5)
        charge2 = Charge(-2, RIGHT*2.5)
        field = ElectricField(charge1, charge2)
        self.add(charge1)
        self.add(charge2)
        self.add(field)
        
        func = lambda p: field.field_func(p)
        stream_lines = StreamLines(func, stroke_width=3, max_anchors_per_line=30)
        self.add(stream_lines)
        stream_lines.start_animation(warm_up=False, flow_speed=1.5)
        self.wait(4)

In [5]:
%%manim -qh -v WARNING MyExampleScene
from manim import *

class MyExampleScene(Scene):         
    def construct(self):
        current1 = Current(LEFT * 2.5)
        current2 = Current(RIGHT * 2.5, direction=IN)
        field = MyField(current1, current2)
        self.add(field, current1, current2)
        
        func = lambda p: field.field_func(p)

#         stream_lines = StreamLines(
#             func,         
#             dt=0.2,
#             virtual_time=10,
#             stroke_width=3,
#             stroke_color=GREY_A,
#             )
    
        stream_lines = StreamLines(func, stroke_width=3, max_anchors_per_line=30)
        self.add(stream_lines)
        stream_lines.start_animation(warm_up=False, flow_speed=1.5)
        self.wait(2)